In [2]:
import time
import numpy as np
import pycuda.autoinit
import pycuda.driver as drv
from pycuda.compiler import SourceModule
from scipy.spatial.distance import cdist

In [3]:
mod_combinations = SourceModule('''
__global__ void generate_combinations2(int *matrix, int n)
{
    int idx = threadIdx.x + blockDim.x * blockIdx.x;
    int tmp = idx;
    for (int i = 0; i < n; i++) {
        matrix[idx * n + i] = tmp % 2;
        tmp /= 2;
    }
}'''
                                )

def generate_combinations_pycuda(n):
    generate_combinations2 = mod_combinations.get_function("generate_combinations2")
    matrix_gpu = np.zeros((2**n, n), dtype=np.int32)
    block_size = 32
    grid_size = (2**n + block_size - 1) // block_size
    generate_combinations2(drv.Out(matrix_gpu), np.int32(n), block=(block_size, 1, 1), grid=(grid_size,1))
    return matrix_gpu

A = B = generate_combinations_pycuda(3)

print(A)

# def calculate_distance2(A,B):
#     return cdist(A,B, metric='euclidean')

# # # Algorithm 1
# def algorithm1():
#     return calculate_distance1(A, B)

# # Algorithm 2
# def algorithm2():
#     return calculate_distance2(A,B)

# # def algorithm3(matrix):
# #     return calculate_distance2(matrix, matrix)
# #     pass

# # Function to measure the runtime of a given algorithm
# def measure_runtime(algorithm):
#     start_time = time.time()
#     algorithm()
#     end_time = time.time()
#     return end_time - start_time

# # Test the algorithms
# runtime1 =[]
# runtime2 = []
# runtime3 = []

# B = np.array([[1, 1, 1], [0, 0, 0], [1,1,1]])


# #print(calculate_distance1(A, B))
# #dist1 = calculate_distance1(A, B)
# dist2 = calculate_distance2(A, B)
# #add distances in same vectors
# dist2 = np.sum(dist2, axis=1)
# print('this is our vector')
# print(A[np.argmax(dist2)])
# #print(dist1)
# print(dist2)
# #print(np.allclose(dist1, dist2))
# #print(calculate_distance2(A, B))

# for i in range(30):
#     #runtime1.append(measure_runtime(algorithm1))
#     runtime2.append(measure_runtime(algorithm2))
#     #runtime3.append(measure_runtime(algorithm3, n))

# print(f"Runtime of Algorithm 1: {np.mean(runtime1)} seconds")
# print(f"Runtime of Algorithm 2: {np.mean(runtime2)} seconds")
# print(f"Runtime of Algorithm 3: {np.mean(runtime3)} seconds")

[[0 0 0]
 [1 0 0]
 [0 1 0]
 [1 1 0]
 [0 0 1]
 [1 0 1]
 [0 1 1]
 [1 1 1]]


In [30]:
def generate_combinations(n):
    # create an empty list to store the combinations
    combinations = []
    
    # generate all possible combinations of 0's and 1's for n items
    for i in range(2**n):
        # convert i to a binary string of length n with leading zeros
        binary_str = bin(i)[2:].zfill(n)
        
        # convert the binary string to a list of integers
        combination = [int(digit) for digit in binary_str]
        
        # append the combination to the list of combinations
        combinations.append(combination)
    return combinations

def fft_bids_init(n):
    ncol = 3  # number of items in value model
    available_bids = generate_combinations(ncol)
    b_0 = available_bids[np.random.randint(0, len(available_bids))]
    D = np.zeros((n, ncol))
    D[0] = b_0

    for i in range(1, n):
        distances = cdist(available_bids, D[:i], metric='euclidean')
        sum_dist = np.sum(distances, axis=1)
        D[i] = available_bids[np.argmax(sum_dist)]

    def myfunc(bundle):
    #generate random value for some bundle
        return np.random.uniform(1, 100)
    D = np.hstack((D, np.apply_along_axis(myfunc, 1, D).reshape(-1, 1)))
    del myfunc
    return (D)


print(fft_bids_init(3))

b_0
[0, 0, 1]
first D
[[0. 0. 1.]
 [0. 0. 0.]
 [0. 0. 0.]]
We test this
[[0. 0. 1.]]
distances
[[1.        ]
 [0.        ]
 [1.41421356]
 [1.        ]
 [1.41421356]
 [1.        ]
 [1.73205081]
 [1.41421356]]
D new
[[0. 0. 1.]
 [1. 1. 0.]
 [0. 0. 0.]]
We test this
[[0. 0. 1.]
 [1. 1. 0.]]
distances
[[1.         1.41421356]
 [0.         1.73205081]
 [1.41421356 1.        ]
 [1.         1.41421356]
 [1.41421356 1.        ]
 [1.         1.41421356]
 [1.73205081 0.        ]
 [1.41421356 1.        ]]
D new
[[0. 0. 1.]
 [1. 1. 0.]
 [0. 0. 0.]]
[[ 0.          0.          1.         92.62611185]
 [ 1.          1.          0.         35.0806653 ]
 [ 0.          0.          0.         69.91865988]]
